<a href="https://colab.research.google.com/github/mikalainis/AI-Agent-Capstone/blob/main/Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai ddgs alpaca-py nest_asyncio slack_sdk

In [ ]:
import os
import json
import warnings
import nest_asyncio
import pandas as pd
from datetime import datetime, timedelta # Added for date calculation
from google.colab import userdata

# 1. Silence Warnings
warnings.filterwarnings("ignore")

# Patch for Colab
nest_asyncio.apply()

# --- LIBRARIES ---
from google import genai
from google.genai import types
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

# NEW: Modern Alpaca SDK Imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.enums import DataFeed

# --- CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    ALPACA_API_KEY = userdata.get('ALPACA_API_KEY')
    ALPACA_SECRET = userdata.get('ALPACA_SECRET')
    # NEW: Slack Configuration
    SLACK_BOT_TOKEN = userdata.get('SLACK_BOT_TOKEN')
    # Updated Channel Name (Slack requires lowercase/no spaces for API calls)
    SLACK_CHANNEL = "D0A1C7TBB5E"
except Exception:
    # Placeholder for safety
    GOOGLE_API_KEY = "YOUR_KEY"
    ALPACA_API_KEY = "YOUR_KEY"
    ALPACA_SECRET = "YOUR_SECRET"
    SLACK_BOT_TOKEN = "YOUR_SLACK_TOKEN"

# --- CLIENTS ---
# 1. Gemini
client = genai.Client(api_key=GOOGLE_API_KEY)

# 2. Alpaca Trading (Executes orders)
alpaca = TradingClient(ALPACA_API_KEY, ALPACA_SECRET, paper=True)

# 3. Alpaca Data (History)
stock_data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET)
crypto_data_client = CryptoHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET)

# 4. Slack Client
slack_client = WebClient(token=SLACK_BOT_TOKEN)

class DarwinianSwarm:
    def __init__(self, ticker: str):
        self.ticker = ticker.upper()
        self.state = {
            "ticker": ticker,
            "news_summary": "",
            "analyst_decision": "HOLD",
            "confidence": 0,
            "reasoning": ""
        }

    # --- AGENT 1: NEWS HUNTER ---
    def fetch_news(self):
        print(f"\n1️⃣  NEWS HUNTER: Searching for {self.ticker}...")
        try:
            print("   🔎 Accessing Google Search Index...")

            search_prompt = f"Find the latest financial news, analyst ratings, and market sentiment for {self.ticker}. Summarize the top 3 headlines."

            response = client.models.generate_content(
                model="gemini-2.5-flash-lite",
                contents=search_prompt,
                config=types.GenerateContentConfig(
                    tools=[types.Tool(google_search=types.GoogleSearch())],
                    response_mime_type="text/plain"
                )
            )

            news_text = response.text
            if not news_text:
                print("   ⚠️ No news found via Google Search.")
                news_text = "No specific news found. Rely on technical trends."
            else:
                print(f"   ✅ News retrieved via Google Grounding.")

            self.state["news_summary"] = news_text
            return news_text

        except Exception as e:
            print(f"   ⚠️ Search Error: {e}")
            self.state["news_summary"] = f"Error fetching news."

    # --- AGENT 2: ANALYST SWARM ---
    def analyze_market(self):
        print(f"2️⃣  GEMINI ANALYST: Reading & Reasoning...")

        response_schema = {
            "type": "OBJECT",
            "properties": {
                "decision": {"type": "STRING", "enum": ["BUY", "SELL", "HOLD"]},
                "confidence": {"type": "INTEGER"},
                "reasoning": {"type": "STRING"}
            }
        }

        prompt = f"""
        You are a Senior Wall Street Analyst.
        Analyze this news for {self.ticker}:
        {self.state['news_summary']}

        Return a JSON decision (BUY/SELL/HOLD) and confidence (0-100).
        """

        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash-lite",
                contents=prompt,
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    response_schema=response_schema
                )
            )
            data = json.loads(response.text)
            self.state["analyst_decision"] = data["decision"]
            self.state["confidence"] = data["confidence"]
            self.state["reasoning"] = data["reasoning"]

            print(f"   ✅ Decision: {data['decision']} ({data['confidence']}%)")
            print(f"   📝 Reason: {data['reasoning']}")
        except Exception as e:
            print(f"   ❌ AI Error: {e}")

    # --- HELPER: RSI STRATEGY ---
    def get_market_data(self):
        """Fetches historical bars, looking back 30 days to handle weekends/holidays"""
        print(f"   📉 Fetching historical data for {self.ticker}...")

        # Check if Crypto
        is_crypto = self.ticker in ["BTCUSD", "ETHUSD", "SOLUSD", "LINKUSD", "DOGEUSD"]

        # FIX: Set start date to 30 days ago to ensure we capture recent trading days
        start_time = datetime.now() - timedelta(days=30)

        try:
            if is_crypto:
                request_params = CryptoBarsRequest(
                    symbol_or_symbols=self.ticker,
                    timeframe=TimeFrame.Hour,
                    start=start_time, # Explicit start to get history
                    limit=200 # Get enough bars
                )
                bars = crypto_data_client.get_crypto_bars(request_params)
            else:
                # Use IEX for free tier stocks
                request_params = StockBarsRequest(
                    symbol_or_symbols=self.ticker,
                    timeframe=TimeFrame.Hour,
                    start=start_time, # Explicit start fixes "No data" on weekends
                    limit=200,
                    feed=DataFeed.IEX
                )
                bars = stock_data_client.get_stock_bars(request_params)

            if not bars.data:
                print("   ⚠️ No data returned. Market might be closed or ticker invalid.")
                return pd.DataFrame()

            df = bars.df
            if isinstance(df.index, pd.MultiIndex):
                df = df.reset_index()

            # Normalize columns to lowercase to prevent KeyError
            df.columns = [c.lower() for c in df.columns]

            # We only need the last 100 rows for RSI calculation
            return df.tail(100)

        except Exception as e:
            print(f"   ⚠️ Data Fetch Error: {e}")
            return pd.DataFrame()

    def calculate_rsi(self, data: pd.DataFrame, period: int = 14):
        """Wilder's RSI Calculation"""
        if 'close' not in data.columns:
            return None

        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)

        # Wilder's Smoothing (alpha = 1/period) -> com = period - 1
        avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()
        avg_loss = loss.ewm(com=period - 1, min_periods=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    # --- NOTIFICATION HELPERS ---
    def notify_slack(self, message):
        """Sends a non-blocking informational message to Slack"""
        if SLACK_BOT_TOKEN and SLACK_BOT_TOKEN != "YOUR_SLACK_TOKEN":
            try:
                slack_client.chat_postMessage(channel=SLACK_CHANNEL, text=message)
                print("   📨 Slack Notification Sent!")
            except SlackApiError as e:
                error = e.response['error']
                # Catch specific errors to reassure user
                if error == 'invalid_auth':
                    print("   ⚠️ Slack Auth Failed: Check your SLACK_BOT_TOKEN in Secrets.")
                elif error == 'missing_scope':
                    print("   ⚠️ Slack Scope Error: Your Bot needs the 'chat:write' permission scope.")
                elif error == 'channel_not_found':
                    print(f"   ⚠️ Slack Channel Error: Bot not in '{SLACK_CHANNEL}'. Invite it with /invite @YourBotName.")
                else:
                    print(f"   ❌ Slack Error: {error}")
        else:
            print("   ⚠️ Slack Token not configured. Message skipped.")

    def request_human_approval(self, decision, current_price, rsi_val):
        """Sends Slack message AND prints to Console, then pauses for input"""
        print(f"\n🛑 HUMAN-IN-THE-LOOP: Requesting approval...")

        rsi_str = f"{rsi_val:.2f}" if rsi_val else "N/A"
        price_str = f"${current_price:.2f}" if current_price else "N/A"

        # 1. Print formatted report to Console (Backup for when Slack fails)
        console_report = (
            f"\n   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓\n"
            f"   ┃  🚨 TRADE AUTHORIZATION REQUESTED      ┃\n"
            f"   ┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫\n"
            f"   ┃  Ticker:     {self.ticker:<22}┃\n"
            f"   ┃  Action:     {decision:<22}┃\n"
            f"   ┃  Price:      {price_str:<22}┃\n"
            f"   ┃  RSI:        {rsi_str:<22}┃\n"
            f"   ┃  Confidence: {self.state['confidence']}%{' '*19}┃\n"
            f"   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛\n"
            f"   📝 Reasoning: {self.state['reasoning']}\n"
        )
        print(console_report)

        # 2. Try sending to Slack
        slack_msg = (
            f"🚨 *Darwinian Agent Trade Alert* 🚨\n"
            f"*Ticker:* {self.ticker}\n"
            f"*Action:* {decision}\n"
            f"*Price:* {price_str}\n"
            f"*RSI:* {rsi_str}\n"
            f"*Confidence:* {self.state['confidence']}%\n"
            f"*Reason:* {self.state['reasoning']}\n\n"
            f"_Please approve in terminal._"
        )
        self.notify_slack(slack_msg)

        # 3. Blocking Wait for User
        print("\n   👉 Check details above (or in Slack).")
        user_input = input(f"   ❓ Type 'YES' to execute {decision} order: ")

        if user_input.strip().upper() == "YES":
            print("   ✅ User Approved.")
            return True
        else:
            print("   ❌ User Denied.")
            return False

    # --- AGENT 4: TRADER ---
    def execute_trade(self):
        print(f"\n3️⃣  TRADER: Checking criteria...")
        decision = self.state["analyst_decision"]
        confidence = self.state["confidence"]

        # 1. Check Decision - HOLD
        if decision == "HOLD":
            msg = (f"⏸️ *Strategy Update: HOLD*\n"
                   f"*Ticker:* {self.ticker}\n"
                   f"*Reason:* {self.state['reasoning']}")
            print("   ⏸️ Holding (Analyst Decision).")
            self.notify_slack(msg)
            return

        # 2. Check Decision - Low Confidence
        if confidence < 75:
            msg = (f"🛑 *Trade Aborted: Low Confidence*\n"
                   f"*Ticker:* {self.ticker}\n"
                   f"*Confidence:* {confidence}% (Threshold: 75%)\n"
                   f"*Reason:* {self.state['reasoning']}")
            print(f"   🛑 Low Confidence ({confidence}%). No trade.")
            self.notify_slack(msg)
            return

        # Market Data placeholders
        current_price = 0.0
        current_rsi = 0.0

        # 3. Check Technicals (RSI) FOR BUY ORDERS
        if decision == "BUY":
            print("   🤖 Consulting RSI Strategy before BUY...")
            df = self.get_market_data()

            if df.empty:
                msg = f"🛑 *Trade Aborted: No Technical Data*\n*Ticker:* {self.ticker}"
                print("   🛑 No Technical Data. Aborting Trade for Safety.")
                self.notify_slack(msg)
                return

            df['rsi'] = self.calculate_rsi(df)
            if df['rsi'] is None:
                msg = f"🛑 *Trade Aborted: RSI Calculation Error*\n*Ticker:* {self.ticker}"
                print("   🛑 RSI Calc Failed. Aborting.")
                self.notify_slack(msg)
                return

            current_rsi = df['rsi'].iloc[-1]
            current_price = df['close'].iloc[-1]

            print(f"   📊 Current RSI: {current_rsi:.2f} | Price: ${current_price:.2f}")

            # RSI Strategy Settings
            RSI_OVERSOLD = 30

            if current_rsi > RSI_OVERSOLD:
                msg = (f"⏳ *RSI Gate: Wait for Dip*\n"
                       f"*Ticker:* {self.ticker}\n"
                       f"*Price:* ${current_price:.2f}\n"
                       f"*RSI:* {current_rsi:.2f} (Needs < 30)\n"
                       f"*Action:* Trade deferred.")
                print(f"   ⏳ RSI GATE: Price ({current_price}) is not oversold (RSI {current_rsi:.1f} > 30). Waiting for dip.")
                self.notify_slack(msg)
                return

            print(f"   ✅ RSI GATE PASSED: Oversold ({current_rsi:.1f}). Buying the dip!")

        # 4. Human Approval
        is_approved = self.request_human_approval(decision, current_price, current_rsi)
        if not is_approved:
            self.notify_slack(f"❌ *Trade Cancelled by User*\n*Ticker:* {self.ticker}")
            return

        # 5. Execution
        print(f"   💸 Executing {decision} for {self.ticker}...")
        try:
            market_order_data = MarketOrderRequest(
                symbol=self.ticker,
                qty=1,
                side=OrderSide.BUY if decision == "BUY" else OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )
            order = alpaca.submit_order(order_data=market_order_data)
            print(f"   ✅ Trade Submitted! Order ID: {order.id}")
            self.notify_slack(f"✅ *Order Filled*\n*ID:* {order.id}\n*Side:* {decision}\n*Ticker:* {self.ticker}")
        except Exception as e:
            print(f"   ❌ Trade Failed: {e}")
            self.notify_slack(f"❌ *Trade Failed Error*\n*Ticker:* {self.ticker}\n*Error:* {e}")

    def run(self):
        self.fetch_news()
        self.analyze_market()
        self.execute_trade()
        print("\n--- DONE ---")

# --- RUN ---
if __name__ == "__main__":
    ticker = input("Enter Ticker (e.g. NVDA): ") or "NVDA"
    bot = DarwinianSwarm(ticker)
    bot.run()